In [ ]:
!pip install autogluon  # autogluon==0.2.0

In [ ]:
# Importing core libraries
import numpy as np
import pandas as pd

# Importing AutoGluon
from autogluon.tabular import TabularDataset, TabularPredictor

# Scikit Learn
from sklearn.model_selection import train_test_split

In [ ]:
# Loading data 
X_train = pd.read_csv("../input/amazon-employee-access-challenge/train.csv")
X_test = pd.read_csv("../input/amazon-employee-access-challenge/test.csv")

# Separating the identifier from the test data
ids = X_test["id"]
X_test.drop("id", axis="columns", inplace=True)

# Converting all integer variables to categorical
integer_cols = X_train.drop('ACTION', axis=1).select_dtypes(include=['int']).columns
X_train[integer_cols] = X_train[integer_cols].astype('str', copy=False)
X_test[integer_cols] = X_test[integer_cols].astype('str', copy=False)

In [ ]:
train_data = TabularDataset(X_train)

In [ ]:
label = 'ACTION'
print("Summary of target variable: \n", train_data[label].describe())

In [ ]:
!mkdir agModels

In [ ]:
save_path = 'agModels'  # specifies folder to store trained models
presets='best_quality'
predictor = TabularPredictor(label=label, path=save_path).fit(train_data,
                                                              presets=presets,
                                                              time_limit=60 * 60 * 8)

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
with open('./agModels/SummaryOfModels.html', 'r') as r:
    plots = '\n'.join(r.readlines())
    
from IPython.core.display import display, HTML
import bokeh
display(HTML(plots))

In [ ]:
test_data = TabularDataset(X_test)
test_preds = predictor.predict_proba(test_data)

In [ ]:
submission = pd.DataFrame({'Id':ids, 'ACTION': test_preds[1].ravel()})
submission.to_csv("submission.csv", index = False)